In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix,roc_auc_score
%matplotlib inline

train = pd.read_csv('cleaned_mostRecStat_train.csv',header=0,encoding='iso-8859-1')
train_y = train['is_placed']
train.drop("is_placed",1)


test = pd.read_csv('cleaned_mostRecStat_test.csv',header=0,encoding='iso-8859-1')
test_y = test['is_placed']
test.drop("is_placed",1)

strings = ['status_name', 'title', 'date_submitted', 'fname', 'lname', 'sourceName', 'gender', 'date_created', 'i9',
          'email', 'address', 'minority', 'ivy','address_field','clarity','personability','zip']
train = train.drop(strings,1)
test = test.drop(strings,1)

train.columns

Index(['is_placed', 'in_process', 'client_id', 'status_id', 'is_rockstar',
       'candidate_id', 'recruiter_id', 'consulting', 'permanent', 'source_id',
       'year_began', 'willingToRelocate', 'is_gmail', 'is_aol', 'is_custom',
       'is_male', 'is_minority', 'is_ivy', 'yearsOfExp', 'is_citizen'],
      dtype='object')

In [2]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

def rocplot(truth, pred, model):
    """This function will roc curve of a given model and return its auc. 
       This function is a slighly augmented copy of the plotUnivariateROC from Lab_FeatureRanking_AUC"""
    fpr, tpr, thresholds = metrics.roc_curve(truth, pred)
    roc_auc = metrics.auc(fpr, tpr)
    
    if roc_auc < 0.5:
        fpr, tpr, thresholds = metrics.roc_curve(truth, -1 * pred)
        roc_auc = metrics.auc(fpr, tpr)
        
    c = (np.random.rand(), np.random.rand(), np.random.rand())
    plt.plot(fpr, tpr, color = c, label = model + ' (AUC = %0.3f)' % roc_auc)
    

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC')
    plt.legend(loc="lower right")
    
    return roc_auc


logreg = LogisticRegression(C = 1)
logreg = logreg.fit(train, train_y)
pred = logreg.predict_proba(test)[:, 1]
rocplot(test_y, pred, "Logistic Regression")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').